In [1]:

from SVFcore_mo import GSVCapture 

gsv = GSVCapture()

gsv.getByLatLong(outdir='img', lat=18.797170700756716, lon=98.96122872758568)

AttributeError: 'GSVCapture' object has no attribute 'input_shape'

In [ ]:
# create class Car

